In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv(r'/Users/allen/Documents/GitHub/ML7331-Lab1/airline_merged.csv',low_memory=False)

In [3]:
#pd.set_option('display.max_columns', None)

In [3]:
#removing unwanted column from previous export
df.pop('Unnamed: 0')

0                0
1                1
2                2
3                3
4                4
            ...   
5819074    5819074
5819075    5819075
5819076    5819076
5819077    5819077
5819078    5819078
Name: Unnamed: 0, Length: 5819079, dtype: int64

In [5]:
df.head()

,year,month,day,day_of_week,airline_cd,airline_name,flight_number,tail_number,origin_airport,airport_origin,city_origin,state_origin,country_origin,latitude_origin,longitude_origin,destination_airport,airport_destination,city_destination,state_destination,country_destination,latitude_destination,longitude_destination,scheduled_departure,departure_time,departure_delay,taxi_out,wheels_off,scheduled_time,elapsed_time,air_time,distance,wheels_on,taxi_in,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled,cancellation_reason,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
0,2015,1,1,4,AS,Alaska Airlines Inc.,98,N407AS,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,American Airlines Inc.,2336,N3KUAA,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,PBI,Palm Beach International Airport,West Palm Beach,FL,USA,26.68316,-80.09559,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,US Airways Inc.,840,N171US,SFO,San Francisco International Airport,San Francisco,CA,USA,37.61900,-122.37484,CLT,Charlotte Douglas International Airport,Charlotte,NC,USA,35.21401,-80.94313,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,American Airlines Inc.,258,N3HYAA,LAX,Los Angeles International Airport,Los Angeles,CA,USA,33.94254,-118.40807,MIA,Miami International Airport,Miami,FL,USA,25.79325,-80.29056,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,Alaska Airlines Inc.,135,N527AS,SEA,Seattle-Tacoma International Airport,Seattle,WA,USA,47.44898,-122.30931,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#selecting 5 busiest airports in US
#airports = ['ATL','DFW','ORD','LAX','DEN']
airports = ['DFW','ATL']

In [5]:
df_air = df.loc[df['origin_airport'].isin(airports)]

In [6]:
df_air.describe()

,year,month,day,day_of_week,flight_number,latitude_origin,longitude_origin,latitude_destination,longitude_destination,scheduled_departure,...,scheduled_arrival,arrival_time,arrival_delay,diverted,cancelled,air_system_delay,security_delay,airline_delay,late_aircraft_delay,weather_delay
count,586387.0,586387.000000,586387.000000,586387.000000,586387.000000,586387.000000,586387.000000,584366.000000,584366.000000,586387.000000,...,586387.000000,577342.000000,576153.000000,586387.000000,586387.000000,106940.000000,106940.000000,106940.000000,106940.000000,106940.000000
mean,2015.0,6.172059,15.675349,3.924858,2143.124518,33.336301,-89.578487,35.532542,-90.360087,1441.053789,...,1573.092447,1550.670914,4.610338,0.002427,0.015026,10.908519,0.065813,21.283514,19.590406,5.326688
std,0.0,3.382615,8.772372,1.996210,1406.473190,0.365962,6.198708,5.307810,13.530677,462.091208,...,491.089079,515.011187,36.534843,0.049202,0.121656,24.422401,1.720925,39.962879,38.528129,28.461893
min,2015.0,1.000000,1.000000,1.000000,3.000000,32.895950,-97.037200,17.701890,-157.922410,140.000000,...,1.000000,1.000000,-80.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2015.0,3.000000,8.000000,2.000000,1173.000000,32.895950,-97.037200,31.327370,-96.851770,1030.000000,...,1159.000000,1148.000000,-12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2015.0,6.000000,16.000000,4.000000,1861.000000,33.640440,-84.426940,35.393090,-86.773110,1445.000000,...,1605.000000,1554.000000,-5.000000,0.000000,0.000000,0.000000,0.000000,7.000000,0.000000,0.000000
75%,2015.0,9.000000,23.000000,6.000000,2659.000000,33.640440,-84.426940,39.871950,-80.943130,1840.000000,...,2006.000000,1959.000000,7.000000,0.000000,0.000000,15.000000,0.000000,26.000000,23.000000,0.000000
max,2015.0,12.000000,31.000000,7.000000,6899.000000,33.640440,-84.426940,61.174320,-64.798560,2359.000000,...,2359.000000,2400.000000,1405.000000,1.000000,1.000000,862.000000,190.000000,1126.000000,1331.000000,1068.000000


In [7]:
#selecting columns that are dtype != numeric
non_numeric = df_air.select_dtypes(exclude = np.number).columns

In [ ]:
#create binary response for logistic regression
df_air['is_delay'] = np.where(df_air['departure_delay'] <= 0,0,1)

In [8]:
#Departure Delay - response
#altered from https://www.statology.org/one-hot-encoding-in-python/

#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'team' column 
encoder_df = pd.DataFrame(encoder.fit_transform(df_air[non_numeric]).toarray())

#return column names from encoder
encoder_df.columns = encoder.get_feature_names_out()

#merge one-hot encoded columns back with original DataFrame
encoded = df_air.join(encoder_df)

In [10]:
encoded.dropna()

,year,month,day,day_of_week,airline_cd,airline_name,flight_number,tail_number,origin_airport,airport_origin,...,state_destination_VT,state_destination_WA,state_destination_WI,state_destination_WV,state_destination_WY,country_destination_USA,cancellation_reason_A,cancellation_reason_B,cancellation_reason_C,cancellation_reason_nan
70,2015,1,1,4,AA,American Airlines Inc.,1057,N3ASAA,DFW,Dallas/Fort Worth International Airport,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
109,2015,1,1,4,EV,Atlantic Southeast Airlines,3268,N14998,ATL,Hartsfield-Jackson Atlanta International Airport,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
124,2015,1,1,4,DL,Delta Air Lines Inc.,1890,N377DA,DFW,Dallas/Fort Worth International Airport,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
203,2015,1,1,4,AA,American Airlines Inc.,72,N5EKAA,DFW,Dallas/Fort Worth International Airport,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
209,2015,1,1,4,AA,American Airlines Inc.,1100,N3GWAA,DFW,Dallas/Fort Worth International Airport,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [12]:
#drop original columns
final_df = encoded.drop(non_numeric, axis=1)

#head(final_df)

In [13]:
#logistic regression code altered from https://www.datacamp.com/tutorial/understanding-logistic-regression-python

feature_cols = final_df.loc[:, final_df.columns != 'is_delay'].columns
X = final_df[feature_cols] # Features
y = final_df.is_delay # Target variable

In [14]:
#split dataset in features and target variable
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)

In [26]:
#imputing 0 into NaN values for logistic regression after OHE
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)
y_train = y_train.fillna(0)
y_test = y_test.fillna(0)

In [36]:
#checking shape of data
print('X_train shape: ',X_train.shape,'\n')
print('X_test shape: ',X_test.shape,'\n')
print('y_train shape: ',y_train.shape,'\n')
print('y_test shape: ',y_test.shape)

X_train shape:  (439790, 4883) 

X_test shape:  (146597, 4883) 

y_train shape:  (439790,) 

y_test shape:  (146597,)


In [ ]:
# instantiate the model (using the default parameters)
logreg = LogisticRegression()

# fit the model with data
logreg.fit(X_train,y_train)

#predict
y_pred=logreg.predict(X_test)

In [ ]:
#Metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))
print("Recall:",metrics.recall_score(y_test, y_pred))

In [ ]:
#Confusion Matrix
class_names=[0,1] # name  of classes
fig, ax = plt.subplots()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)

# create heatmap
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

Text(0.5,257.44,'Predicted label')

In [ ]:
#ROC Curve
y_pred_proba = logreg.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="Base Logistic Regression, auc= "+str(auc))
plt.legend(loc=4)
plt.show()